产品净值分析

In [1]:
import numpy as np
import os
import tushare as ts
import pandas as pd
import xlrd
import empyrical as ep
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings("ignore")
from pyfolio.timeseries import gen_drawdown_table, perf_stats

plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']

# 读入产品数据

- excel格式，一个sheet对应一个产品
- sheet格式，第一列为日期，第二列为累计净值

In [2]:
file_path = r'./同类产品表.xlsx'
excel = xlrd.open_workbook(file_path)
sheet_list = excel.sheet_names()
df_list = []
for each in sheet_list:
    df = pd.read_excel(file_path, each, header=0)
    if df.empty:
        continue
    df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
    df.set_index(df.iloc[:, 0].name, drop=True, inplace=True)
    df = df.resample('D').asfreq()
    df_list.append(df)
total_raw = pd.concat(df_list, axis=1)

total = total_raw.copy()
trade_list = ts.pro_api().trade_cal(exchange='SSE', start_date=total.index.min().strftime('%Y%m%d'),
                                    end_date=total.index.max().strftime('%Y%m%d'))
trade_list = trade_list[trade_list['is_open'] == 1]  # 交易日
total = total[total.index.isin(pd.to_datetime(trade_list['cal_date']))]
total = total.apply(pd.Series.interpolate)

In [3]:
total

,圭源蒙玺启航,纯达二期
日期,,
2017-07-18,1.000000,NaN
2017-07-19,1.000667,NaN
2017-07-20,1.001333,NaN
2017-07-21,1.002000,NaN
2017-07-24,1.001200,NaN
2017-07-25,1.000400,NaN
2017-07-26,0.999600,NaN
2017-07-27,0.998800,NaN
2017-07-28,0.998000,NaN


# 计算日收益率

In [4]:
def value_to_returns(se):
    return se.diff() / se
returns = total.apply(value_to_returns)

# 表现统计

In [5]:
def stats_se(se):
    stats = perf_stats(se.dropna())
    # 剔除不在时段内的
    if total_raw[se.name].dropna().index.max()<se.dropna().index.min():
        return None
    stats['VaR'] = np.percentile(se.dropna(), 100 * 0.05)
    stats = stats.append(gen_drawdown_table(se.dropna(), top=1).iloc[0, :])
    return stats

def stats_df(df):
    result = df.apply(stats_se)
    result['mean'] = result.T.mean()
    result['max_min'] = result.apply(lambda x: x.max() - x.min(), axis=1)
    return result

指标说明
> 列：每列为每个产品，mean表示按行计算指标均值，max_min表示按行计算指标的最大值减最小值

> 行：
- Annual return 年化收益率
- Cumulative returns累计收益率
- Annual volatility年化波动率
- Sharpe ratio夏普比率
- Calmar ratio卡玛比率
- Stability稳定性（净值线性回归的R2）
- Max drawdown最大回撤
- Omega ratio
- Sortino ratio
- Skew
- Daily value at risk（日收益率均值减去两倍标准差）
- VaR 在险价值（日收益率的5%分位数）
- Net drawdown in %	最大回撤百分比
- Peak date 回撤前高点日期
- Valley date 最大回撤日期
- Recovery date 净值恢复日期
- Duration	 持续天数

## 同类产品的统计

In [6]:
use_returns = returns.copy()
result = stats_df(use_returns)
result.to_csv('./out_stats/同类产品的统计.csv', encoding='gbk')
result

,圭源蒙玺启航,纯达二期,mean,max_min
Annual return,0.159313,0.567044,0.363179,0.407731
Cumulative returns,0.37511,0.294938,0.335024,0.0801715
Annual volatility,0.0507097,0.0682951,0.059502,0.0175854
Sharpe ratio,2.94118,6.61685,4.779015,3.67568
Calmar ratio,4.1938,11.5287,7.861275,7.33495
Stability,0.909442,0.955857,0.932649,0.0464151
Max drawdown,-0.0379878,-0.0491852,-0.043586,0.0111974
Omega ratio,1.97858,3.785,2.881788,1.80643
Sortino ratio,7.51504,18.445,12.980013,10.9299
Skew,2.81446,0.708836,1.761646,2.10562


## 近3年同类产品的统计

In [7]:
use_returns = returns[(returns.index.max()-relativedelta(years=3)).strftime('%Y-%m-%d'):]
result = stats_df(use_returns)
result.to_csv('./out_stats/同类产品的统计近3年.csv', encoding='gbk')
result

,圭源蒙玺启航,纯达二期,mean,max_min
Annual return,0.159313,0.567044,0.363179,0.407731
Cumulative returns,0.37511,0.294938,0.335024,0.0801715
Annual volatility,0.0507097,0.0682951,0.059502,0.0175854
Sharpe ratio,2.94118,6.61685,4.779015,3.67568
Calmar ratio,4.1938,11.5287,7.861275,7.33495
Stability,0.909442,0.955857,0.932649,0.0464151
Max drawdown,-0.0379878,-0.0491852,-0.043586,0.0111974
Omega ratio,1.97858,3.785,2.881788,1.80643
Sortino ratio,7.51504,18.445,12.980013,10.9299
Skew,2.81446,0.708836,1.761646,2.10562


## 近1年同类产品的统计

In [8]:
use_returns = returns[(returns.index.max()-relativedelta(years=1)).strftime('%Y-%m-%d'):]
result = stats_df(use_returns)
result.to_csv('./out_stats/同类产品的统计近1年.csv', encoding='gbk')
result

,圭源蒙玺启航,纯达二期,mean,max_min
Annual return,0.157554,0.567044,0.362299,0.40949
Cumulative returns,0.15219,0.294938,0.223564,0.142748
Annual volatility,0.0528002,0.0682951,0.060548,0.0154949
Sharpe ratio,2.79799,6.61685,4.707420,3.81887
Calmar ratio,4.69584,11.5287,8.112296,6.83291
Stability,0.7367,0.955857,0.846279,0.219157
Max drawdown,-0.0335518,-0.0491852,-0.041368,0.0156334
Omega ratio,1.67304,3.785,2.729019,2.11196
Sortino ratio,6.07291,18.445,12.258949,12.3721
Skew,1.53569,0.708836,1.122262,0.826851


# 同类产品合成

## 等权法

- 计算原始数据，有缺失值的产品不参与加权。

In [9]:
new_returns = returns.copy()
new_returns['new'] = returns.mean(axis=1)
new_value = new_returns.cumsum()+1.0
new_value.to_csv('./out_new/同类产品等权合成.csv', encoding='gbk')

## 加权法

- 设置权重，有缺失值的产品不参与加权，并重新将参与加权的产品权重归一化，保持产品间比例不变。
- weight与returns数据结构保持一致。

In [10]:
# 设置权重
weight = [0.5, 0.3, 0.2]
assert len(weight)==len(returns.columns)

AssertionError: 

In [ ]:
def weight_se(se, weight):
    if se.dropna().empty:
        return np.nan
    # na位置
    temp = pd.Series(weight, index=se.index)
    use_se = se[~se.isnull()]
    use_temp = temp[~se.isnull()]
    use_temp = use_temp/use_temp.sum()
    return np.matmul(use_se, use_temp).sum()

new_returns = returns.copy()
new_returns['new'] = new_returns.apply(weight_se, axis=1, args=(weight,))
new_value = new_returns.cumsum()+1.0
new_value.to_csv('./out_new/同类产品加权合成.csv', encoding='gbk')